# Pathfinder for Hidden connections 

Some text

In [1]:
#Imports
from neo4j import GraphDatabase, basic_auth
import neo4jupyter
import pandas as pd

In [2]:
#Create db connector
login = open("Login.txt", "r")
uri = login.readline()
user = login.readline()
password = "XXX"
login.close()

driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
session = driver.session(database="neo4j")

## Finding hidden path

In [ ]:
#Find path between Person and Movie

In [ ]:
#Find path between Person and Person which are not related at the first glance

In [ ]:
#Find path between Movie and Movie which are not related at the first glance

## In which movies are n charactes the same

In [3]:
#Close all connections
driver.close()
session.close()
drive = None
session = None